# **EDA (Exploratory Data Analysis)** de `The Movies Dataset`

## Data Collection

- Cargar los datos
- Revisar los registros


### **Cargar los datos ([`The Movies Dataset`](https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset))**


In [ ]:
import zipfile
import os
# descarmaos usando el CLI de kaggle
!kaggle datasets download -d rounakbanik/the-movies-dataset -p data/

# descomprimimos el archivo
with zipfile.ZipFile("./data/the-movies-dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("./data")
    
# removemos el archivo zip
os.remove("./data/the-movies-dataset.zip")

# removemos los datasets miniatura<
os.remove("./data/links_small.csv")
os.remove("./data/ratings_small.csv")

No se usaran los dataset `ratings_small.csv` (100k) y `links_small.csv` (9k), debido a que solo son versiones mas pequeñas de `ratings.csv` (25M) y `links.csv` (45k)

Leemos con pandas cada uno de los datasets


In [ ]:
import pandas as pd

df_credits = pd.read_csv("./data/credits.csv")
df_keywords = pd.read_csv("./data/keywords.csv")
df_links = pd.read_csv("./data/links.csv")
df_movies_metada = pd.read_csv("./data/movies_metadata.csv")
df_ratings = pd.read_csv("./data/ratings.csv")

### **Revisar los registros**


In [ ]:
# cambiar ancho de output del notebook
pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", False)
pd.set_option("max_colwidth", 80)

#### `credits.csv`


In [ ]:
df_credits.info()

In [ ]:
df_credits.head(5)

In [ ]:
# Observamos que la columna "cast" es un string que contiene una lista de diccionarios
df_credits["cast"][:5]

In [ ]:
# Observamos que la columna "crew" es un string que contiene una lista de diccionarios
df_credits["crew"][:5]

#### `keywords.csv`


In [ ]:
df_keywords.info()

In [ ]:
df_keywords.head(5)

In [ ]:
# Observamos que la columna "keywords" es un string que contiene una lista de diccionarios
df_keywords["keywords"][:5]

#### `links.csv`


In [ ]:
df_links.info()

In [ ]:
df_links.head()

#### `movies_metadata.csv`


In [ ]:
df_movies_metada.info()

In [ ]:
df_movies_metada.head(2)

#### `ratings.csv`


In [ ]:
df_ratings.info()

In [ ]:
df_ratings.head(5)

## Data Exploration

- Revisar los tipos de datos
- Revisar los valores nulos
- Revisar los valores duplicados
- Revisar las estadísticas básicas


Creamos la funcion para saber que columnas tienen valores nulos


In [ ]:
def find_columns_null(df):
    columns_null = df.columns[df.isnull().any()].tolist()
    return columns_null

#### `credits.csv`


In [ ]:
# Revisamos la cantidad de columnas y filas
df_credits.shape

In [ ]:
# Revisamos los tipos de datos
df_credits.dtypes

In [ ]:
# Revisamos los valores unicos
df_credits.nunique()

In [ ]:
# Revisamos los valores nulos
df_credits.isnull().sum()

In [ ]:
# Revisamos las datos duplicados
df_credits.duplicated().sum()

#### `keywords.csv`


In [ ]:
# Revisamos la cantidad de columnas y filas
df_keywords.shape

In [ ]:
# Revisamos los tipos de datos
df_keywords.dtypes

In [ ]:
# Revisamos los valores unicos
df_keywords.nunique()

In [ ]:
# Revisamos los valores nulos
df_keywords.isnull().sum()

In [ ]:
# Revisamos las datos duplicados
df_keywords.duplicated().sum()

#### `links.csv`


In [ ]:
# Revisamos la cantidad de columnas y filas
df_links.shape

In [ ]:
# Revisamos los tipos de datos
df_links.dtypes

In [ ]:
# Revisamos los valores unicos
df_links.nunique()

In [ ]:
# Revisamos los valores nulos
df_links.isnull().sum()

In [ ]:
# Revisamos las datos duplicados
df_links.duplicated().sum()

#### `movies_metadata.csv`


In [ ]:
# Revisamos la cantidad de columnas y filas
df_movies_metada.shape

In [ ]:
# Revisamos los tipos de datos
df_movies_metada.dtypes

In [ ]:
# Revisamos las columnas numericas
df_movies_metada.select_dtypes(include=["int64", "float64"]).describe()

In [ ]:
# Revisamos los valores unicos
df_movies_metada.nunique()

In [ ]:
# Revisamos los valores nulos
df_movies_metada.isnull().sum()

In [ ]:
find_columns_null(df_movies_metada)

In [ ]:
# Revisamos las datos duplicados
df_movies_metada.duplicated().sum()

#### `ratings.csv`


In [ ]:
# Revisamos la cantidad de columnas y filas
df_ratings.shape

In [ ]:
# Revisamos los tipos de datos
df_ratings.dtypes

In [ ]:
# Revisamos los valores unicos
df_ratings.nunique()

In [ ]:
# Revisamos los valores nulos
df_ratings.isnull().sum()

In [ ]:
# Revisamos las datos duplicados
df_ratings.duplicated().sum()

## Data Preprocessing

- Eliminar columnas innecesarias
- Eliminar registros con valores nulos
- Eliminar registros duplicados
- Convertir los tipos de datos


Creamos una funcion para limpiar los datos


In [ ]:
from ast import literal_eval
from typing import Callable

# funcion para obtener los valores de un string
def extract_info(text, obj: str = "name", func: Callable = lambda _: True):
    # Si el texto es un string sin nada que evaluar
    try:
        text = literal_eval(text)
    except:
        return text
    # Si el texto es un diccionario
    if type(text) == dict:
        return text[obj]
    # Si el texto es una lista
    if type(text) == list:
        if len(text) == 0: # Si la lista esta vacia
            return "No"
        if len(text) == 1: # Si la lista tiene un solo elemento
            for i in text:
                return i[obj]
        else: # Si la lista tiene mas de un elemento
            s = [] # lista para almacenar los valores
            for i in text:
                if func(i):  # si la funcion es verdadera
                    s.append(i[obj]) # agregamos el valor a la lista
            return ", ".join(s) # retornamos la lista como un string

#### `credits.csv`


In [ ]:
df_credits.shape

In [ ]:
df_credits.head(2)

Primero eliminamos los valores duplicados, ya que nulos no existen


In [ ]:
df_credits = df_credits.drop_duplicates()
df_credits.shape

Obtenemos `character`, `actors`, `director` de las columnas `cast` y `crew` en dict para poder revisar la data


In [ ]:
df_credits["characters"] = df_credits["cast"].apply(
    extract_info, obj="character"
)  # personajes
df_credits["actors"] = df_credits["cast"].apply(extract_info)  # actores

# obtenemos el director
df_credits["director"] = df_credits["crew"].apply(
    extract_info, args=("name", lambda x: x["job"] == "Director")
)

# obtenemos los guionistas
df_credits["crew"] = df_credits["crew"].apply(
    extract_info, args=("name", lambda x: x["job"] != "Director")
)

Reorganizamos las columnas


In [ ]:
# Removemos las columnas innecesarias
df_credits = df_credits.drop(columns=["cast"])

# Cambiamos el orden de las columnas
df_credits = df_credits[["id", "characters", "actors", "director", "crew"]]

# Cambiamos el nombre de la columna id
df_credits = df_credits.rename(columns={"id": "movie_id"})

In [ ]:
df_credits.head(2)

In [ ]:
# Guardamos el dataset limpio
df_credits.to_csv("./data_clean/credits.csv", index=False)

#### `keywords.csv`


In [ ]:
# Revisamos la cantidad de columnas y filas
df_keywords.shape

In [ ]:
df_keywords.head(2)

Ahora eliminamos los valores duplicados, ya que nulos no existen


In [ ]:
df_keywords = df_keywords.drop_duplicates()
df_keywords.shape

Obtenemos las `keywords` en dict para poder revisar la data


In [ ]:
df_keywords["keywords"] = df_keywords["keywords"].apply(extract_info)

Reorganizamos las columnas


In [ ]:
df_keywords.rename(columns={"id": "movie_id"}, inplace=True)

In [ ]:
df_keywords.head(2)

In [ ]:
# Guardamos el dataset limpio
df_keywords.to_csv("./data_clean/keywords.csv", index=False)

#### `links.csv`


In [ ]:
df_links.shape

In [ ]:
df_links.dtypes

Primero convertimos `tmdbId` de float64 a int64


In [ ]:
df_links["tmdbId"] = df_links["tmdbId"].astype("Int64")
df_links.dtypes

Ahora procesaremos los valores nulos, ya que duplicados no existen


In [ ]:
df_links[df_links["tmdbId"].isnull()]

In [ ]:
df_links["tmdbId"] = df_links["tmdbId"].fillna(-1)

In [ ]:
df_links.isnull().sum()

Ahora renombremos las columnas


In [ ]:
df_links.rename(columns={"movieId": "movie_id"}, inplace=True)

In [ ]:
df_links.head(2)

In [ ]:
# Guardar el dataset limpio
df_links.to_csv("./data_clean/links.csv", index=False)

#### `movies_metadata.csv`


In [ ]:
df_movies_metada.shape

In [ ]:
df_movies_metada.head(2)

Antes de nada eliminamos unos datos raro en el datset


In [ ]:
df_movies_metada[df_movies_metada["belongs_to_collection"] == "0.065736"]

In [ ]:
df_movies_metada[df_movies_metada["belongs_to_collection"] == "2.185485"]

In [ ]:
df_movies_metada[df_movies_metada["belongs_to_collection"] == "1.931659"]

In [ ]:
index_corrupted = df_movies_metada[
    df_movies_metada["belongs_to_collection"] == "0.065736"
].index
df_movies_metada = df_movies_metada.drop(index_corrupted)

index_corrupted = df_movies_metada[
    df_movies_metada["belongs_to_collection"] == "2.185485"
].index
df_movies_metada = df_movies_metada.drop(index_corrupted)

index_corrupted = df_movies_metada[
    df_movies_metada["belongs_to_collection"] == "1.931659"
].index
df_movies_metada = df_movies_metada.drop(index_corrupted)

Primero seleccionaremos que columnas seran necesarias para nuestro caso


In [ ]:
df_movies_metada.drop(
    columns=[
        "imdb_id",
        "homepage",
        "poster_path",
        "video",
        "title",
        "production_countries",
    ],
    inplace=True,
)

Ahora procesaremos los datos nulos de cada columna (Categorica)


In [ ]:
find_columns_null(df_movies_metada)

In [ ]:
# belongs_to_collection
df_movies_metada["belongs_to_collection"].fillna(
    "{'name': 'No'}", inplace=True)
df_movies_metada["belongs_to_collection"] = df_movies_metada[
    "belongs_to_collection"
].apply(extract_info)

In [ ]:
# genres
df_movies_metada["genres"].fillna("No", inplace=True)
df_movies_metada["genres"] = df_movies_metada["genres"].apply(extract_info)

In [ ]:
# production_companies
df_movies_metada["production_companies"].fillna("No", inplace=True)
df_movies_metada["production_companies"] = df_movies_metada[
    "production_companies"
].apply(extract_info)

In [ ]:
# spoken_languages
df_movies_metada["spoken_languages"].fillna("No", inplace=True)
df_movies_metada["spoken_languages"] = df_movies_metada["spoken_languages"].apply(extract_info)

In [ ]:
# tag_line
df_movies_metada["tagline"].fillna("No", inplace=True)

In [ ]:
# status
df_movies_metada["status"].fillna("No", inplace=True)

In [ ]:
# original_language
df_movies_metada["original_language"].fillna("No", inplace=True)

In [ ]:
# overview
df_movies_metada["overview"].fillna("No", inplace=True)

In [ ]:
find_columns_null(df_movies_metada)

In [ ]:
df_movies_metada.head(2)

Ahora procesaremos los datos nulos de cada columna (Numericas)


In [ ]:
find_columns_null(df_movies_metada)

In [ ]:
df_movies_metada[find_columns_null(df_movies_metada)].dtypes

In [ ]:
# popularity
df_movies_metada["popularity"] = df_movies_metada["popularity"].astype(
    "float"
)  # convert to float

df_movies_metada["popularity"].fillna(
    df_movies_metada["popularity"].median(), inplace=True
)  # clean the nulls

In [ ]:
# release_date
df_movies_metada["release_date"] = pd.to_datetime(
    df_movies_metada["release_date"], errors="coerce"
)  # convert to datetime


df_movies_metada["release_date"].fillna(
    df_movies_metada["release_date"].mode()[0], inplace=True
)  # clean nulls

In [ ]:
# revenue
df_movies_metada["revenue"] = df_movies_metada["revenue"].astype(
    "float"
)  # convert to float

df_movies_metada["revenue"].fillna(
    df_movies_metada["revenue"].median(), inplace=True
)  # clean nulls

In [ ]:
# runtime
df_movies_metada["runtime"] = df_movies_metada["runtime"].astype(
    "float"
)  # convert to float

df_movies_metada["runtime"].fillna(
    df_movies_metada["runtime"].mean(), inplace=True
)  # clean nulls

In [ ]:
# vote_average
df_movies_metada["vote_average"].fillna(
    df_movies_metada["vote_average"].mean(), inplace=True
)  # clean nulls

df_movies_metada["vote_average"] = df_movies_metada["vote_average"].astype(
    "float"
)  # convert to float

In [ ]:
# vote_count
df_movies_metada["vote_count"].fillna(
    df_movies_metada["vote_count"].median(), inplace=True
)  # clean nulls
df_movies_metada["vote_count"] = df_movies_metada["vote_count"].astype(
    "int64"
)  # convert to int

In [ ]:
find_columns_null(df_movies_metada)

Ordenamos las columnas


In [ ]:
# ORdenamos las columnas
df_movies_metada = df_movies_metada[
    [
        "id",
        "adult",
        "belongs_to_collection",
        "budget",
        "genres",
        "original_language",
        "overview",
        "popularity",
        "production_companies",
        "release_date",
        "revenue",
        "runtime",
        "spoken_languages",
        "status",
        "tagline",
        "vote_average",
        "vote_count",
    ]
]

df_movies_metada.rename(columns={"id": "movie_id"}, inplace=True)

In [ ]:
df_movies_metada.head(1)

In [ ]:
# guardar el dataset limpio
df_movies_metada.to_csv("./data_clean/movies_metadata.csv", index=False)

#### `ratings.csv`

In [ ]:
df_ratings.shape

In [ ]:
df_ratings.head(2)

In [ ]:
df_ratings.rename(columns={"movieId": "movie_id",
                  "userId": "user_id"}, inplace=True)

In [ ]:
df_ratings.head(2)

In [ ]:
# guardar el dataset limpio
df_ratings.to_csv("./data_clean/ratings.csv", index=False)

#### `data_clean.csv`

In [ ]:
df_clean = pd.merge(df_movies_metada, df_ratings, on="movie_id")

## Data Visualization

- Visualizar los datos
- Visualizar la correlación


In [ ]:
import random

def generate_rgb():
    # Generar valores RGB aleatorios
    r = random.randint(0, 255)
    g = random.randint(0, 255)
    b = random.randint(0, 255)

    # Convertir los valores RGB a formato hexadecimal
    color_hex = "#{:02x}{:02x}{:02x}".format(r, g, b)

    return color_hex

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from collections import Counter

In [ ]:
df_movies_metada["movie_id"] = df_movies_metada["movie_id"].astype("int64")
df = df_movies_metada.merge(df_keywords, on="movie_id").merge(
    df_credits, on="movie_id")
df["budget"] = df["budget"].astype(float)
df["revenue"] = df["revenue"].astype(float)

In [ ]:
plt.figure(figsize=(18, 18))  # tamaño de la figura
plt.title(
    "Las palabras mas comunes en las descripciones de las peliculas\n",
    fontsize=28,
    weight=600,
    color="#1f1f1f",
)
# Generar la nube de palabras
wordcloud = WordCloud(max_words=1000, min_font_size=10, height=650, width=1500).generate(
    " ".join(df["overview"])
)

# Mostrar la nube de palabras
plt.imshow(wordcloud)

Es interesante notar que palabras como "life", "one", "find", "love", "story" y "world" son recurrentes en las descripciones de las películas. La frecuencia de estas palabras podría indicar que estas temáticas son centrales en muchas películas del conjunto de datos.

In [ ]:
plt.figure(figsize=(18, 18))  # tamaño de la figura
plt.title(
    "Las palabras clave mas comunes en las peliculas\n",
    fontsize=28,
    weight=600,
    color="#1f1f1f",
)
wordcloud = WordCloud(
    max_words=1000, min_font_size=10, height=650, width=1500
).generate(" ".join(df["keywords"]))

plt.imshow(X=wordcloud)

Se puede observar que palabras como "director", "woman", "independent film", "based" y "highschool" son frecuentes en las palabras clave de las películas. Estas palabras indican temas y características específicas que son relevantes en el contexto del cine. Por ejemplo, "director" sugiere la importancia del rol del director en la película, "woman" podría indicar la presencia de personajes femeninos destacados, "independent film" señala películas producidas de manera independiente fuera de los estudios principales, "based" podría referirse a películas basadas en eventos reales o libros, y "highschool" indica historias ambientadas en escuelas secundarias. Estos temas y elementos pueden ser importantes para segmentar y comprender mejor las películas en términos de género, audiencia objetivo y estilo cinematográfico.

In [ ]:
# Generar un grafico de barras con los generos mas comunes
sns.displot(
    data=df,
    x="release_date",
    kind="hist",
    kde=True,
    facecolor=generate_rgb(),
    edgecolor="#f1f1f1",
    line_kws={"lw": 1.5},
    color="red",
    aspect=4,
)
plt.title(
    "Cantidad de peliculas estrenadas por año", fontsize=16, weight=700, color="#1f1f1f"
)

Desde 1930, la industria cinematográfica ha experimentado un crecimiento significativo durante los últimos 50 años. Asimismo, la disminución en el número total de películas estrenadas alrededor de 2020 puede atribuirse al hecho de que el conjunto de datos solo contiene datos limitados de esos años .

In [ ]:
# Preparamos los datos
genres_list: list[str] = df["genres"].apply(lambda x: x.split(", ")).sum()
colors: list[str] = [
    generate_rgb(),
    generate_rgb(),
    generate_rgb(),
    generate_rgb(),
    generate_rgb()
]

# Organizamos los datos
df_top = pd.DataFrame(Counter(genres_list).most_common(5), columns=["genre", "total"])
df_full = pd.DataFrame([Counter(genres_list)]).transpose().sort_values(by=0, ascending=False)  # type: ignore

# Ahora graficamos el grafico de barras de las top 5
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14, 6))

# Grafico de barras
ax = sns.barplot(
    data=df_top,
    x="genre",
    y="total",
    ax=axes[0],
    palette=colors,
    hue="genre",
    legend=False
)
ax.set_title("Top 5 Generos de Películas", fontsize=16, weight=600, color="#1f1f1f")
sns.despine() # removemos los bordes

# Ahora graficamos el grafico circular de las top 5 y otros
df_top.loc[len(df_top)] = {"genre": "Otros", "total": df_full[6:].sum()[0]}  # type: ignore
plt.title(
    "Porcentaje de géneros de películas", fontsize=16, weight=600, color="#1f1f1f"
)

colors.insert(0, generate_rgb()) # insertamos un color para "Otros"
axes[1].pie(
    x=df_top["total"],
    labels=df_top["genre"],
    autopct="%1.1f%%",
    textprops=dict(color="#1f1f1f", fontsize=12),
    explode=[0, 0, 0, 0, 0, 0.1],
    colors=colors
)
axes[1].axis("off")

El género drama destaca como el más dominante, con más de 20,000 películas en el conjunto de datos.Aunque los cinco géneros principales son prominentes, todavía existen numerosos géneros adicionales en el conjunto de datos, representando el 38.8% del total de géneros cinematográficos.

In [ ]:
# Con relplot graficamos la relacion entre Rating y Popularidad en base al voto
sns.relplot(
    data=df,
    x="vote_average",
    y="popularity",
    size="vote_count",
    sizes=(30, 190),
    alpha=0.65,
    aspect=4,
    color=generate_rgb(),
    legend="brief",
)
# Agregamos el titulo
plt.title(
    "La relacion entre Rating y Popularidad",
    fontsize=16,
    weight=600,
    color="#1f1f1f",
)

- Las películas que obtuvieron una calificación de 0 o 10 se deben básicamente a un pequeño número de votantes. A medida que aumenta el recuento de votos, lo más probable es que la calificación esté entre 5 y 8,5.
- Está claro que las películas populares obtendrán más votos como se muestra en la trama anterior.

In [ ]:
# Definimos el tamaño de la figura
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))

# Agregamos el titulo
plt.suptitle(
    "La influencia del presupuesto y los ingresos en la popularidad de las películas",
    fontsize=16,
    weight=600,
    color="#1f1f1f",
)

# Graficamos la relacion entre el presupuesto y la popularidad
sns.regplot(
    data=df,
    x="budget",
    y="popularity",
    scatter_kws={"color": generate_rgb(), "alpha": 0.5, "s": 25},
    line_kws={"color": "red", "lw": 1.5},
    ax=axes[0],
)

# Graficamos la relacion entre los ingresos y la popularidad
sns.regplot(
    data=df,
    x="revenue",
    y="popularity",
    scatter_kws={"color": generate_rgb(), "alpha": 0.5, "s": 25},
    line_kws={"color": "red", "lw": 1.5},
    ax=axes[1],
)

plt.tight_layout()

El presupuesto y los ingresos influyen ligeramente en la popularidad de las películas.

In [ ]:
plt.figure(figsize=(12, 10)) #  tamaño de la figura
plt.title(
    "Correlacion de las caracteristicas de las peliculas\n",
    fontsize=18,
    weight=600,
    color="#1f1f1f",
)
# Generamos el mapa de calor
sns.heatmap(
    df.select_dtypes(include=["int64", "float64"]).corr(),  # seleccionamos las columnas numericas
    annot=True, # mostrar los valores
    cmap="viridis", # mapa de colores
    fmt=".2f", # formato de los valores
    linewidths=1, # ancho de las lineas
    linecolor="white", # color de las lineas
)


El recuento de votos, el presupuesto y la popularidad son tres características dominantes que pueden influir significativamente en los ingresos de las películas. Estos factores suelen estar correlacionados con el éxito financiero de una película.